In [1]:
from collections import Counter
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from ast import literal_eval
import json

In [2]:
file_name = "9M-5-[1-2001]-[a]-lcm.out"
output_file = f"queries_results/python_{file_name.replace('out','csv')}"

In [3]:
encoder = LabelEncoder()
encoder.classes_ = np.load(f'../../plots/encoders/{file_name}.npy')

def partial_df():
    # Groupes over two periods only
    links = pd.read_csv(f"../../plots/links/{file_name}")
    links["user_id"] = links["user_id"].apply(lambda x : x.split()).apply(set)
    groups = pd.read_csv(f"../../plots/groups/{file_name}",index_col=0)
    groups.drop(["itemset_name","property_values"],axis=1,inplace=True)
    groups = groups.drop(['depth',"a","size","support",],axis=1)
    groups["itemsets"] = groups["itemsets"].apply(lambda x :set(encoder.inverse_transform([int(i) for i in x.split()])))
    groups["user_ids"] =  groups["user_ids"].apply(literal_eval)
    return links.merge(groups.reset_index(),left_on="source",right_on='index').merge(groups.reset_index(),left_on="target",right_on='index')

def full_df():
    # Groups over three periods only
    links = pd.read_csv(f"../../plots/links/{file_name}")
    links["user_id"] = links["user_id"].apply(lambda x : x.split()).apply(set)
    groups = pd.read_csv(f"../../plots/groups/{file_name}",index_col=0)
    groups.drop(["itemset_name","property_values"],axis=1,inplace=True)
    groups = groups.drop(['depth',"a","size","support",],axis=1)
    groups["itemsets"] = groups["itemsets"].apply(lambda x :set(encoder.inverse_transform([int(i) for i in x.split()])))
    groups["user_ids"] =  groups["user_ids"].apply(literal_eval).apply(set)
    
    groups = groups.reset_index() 
    links = links.merge(links,left_on="target",right_on="source")
    links = links.merge(groups[["itemsets","index"]],left_on="source_x",right_on="index")
    links = links.merge(groups[["itemsets","index"]],left_on="target_x",right_on="index")
    links = links.merge(groups[["itemsets","index"]],left_on="target_y",right_on="index")
    links.drop(["source_y","index","index_x","index_y"],axis=1,inplace=True)
    links = links.rename(columns={"itemsets":"itemsets_z","target_y":"target_z"})
    links["user_id_z"] = links.apply(lambda x : list(x["user_id_x"].intersection(x["user_id_y"])),axis=1)
    links = links[links.user_id_z.apply(len)>0]
    return links

def promotion_query(x,query_name):
    if x["period_x"] == before_promo_period and x["period_y"]== promo_period: # _x => T-1 , _y => T 
        if query_name =="adoption":
            return x["itemsets_y"].difference(x["itemsets_x"])
        if query_name =="loyalty1":
            return x["itemsets_y"].intersection(x["itemsets_x"])
    if x["period_x"]== promo_period and x["period_y"]== after_promo_period: # _x => T , _y => T+1 
        if query_name =="loyalty2":
            return x["itemsets_y"].intersection(x["itemsets_x"])
    return set()

with open("../../datasets/Total/ids_mapping.json","r") as file:
    mapping = json.load(file)



# Product adoption & Loyalty

In [13]:
df = partial_df()
df.head(2)

,source,target,user_id,index_x,user_ids_x,itemsets_x,period_x,index_y,user_ids_y,itemsets_y,period_y
0,0,783,"{129151,131514,341101,348948,354730,381808,407...",0,"[100169, 100882, 102233, 103922, 105498, 10564...",{3425909000482},2018-09-01,783,"[103316, 107231, 110754, 113725, 113895, 11444...",{3425909002011},2018-12-01
1,9,783,{470752},9,"[146720, 249738, 250022, 470752, 534467, 10788...",{8711500406071},2018-09-01,783,"[103316, 107231, 110754, 113725, 113895, 11444...",{3425909002011},2018-12-01


T-1, T and T+1

In [14]:
before_promo_period = df.period_x.min()
after_promo_period = df.period_y.max()
promo_period = list(set(df.period_x.unique()).difference((before_promo_period,after_promo_period)))[0]
print(f"T-1 : {before_promo_period}")
print(f"T : {promo_period}")
print(f"T+1 : {after_promo_period}")

T-1 : 2018-09-01
T : 2018-12-01
T+1 : 2019-02-01


# Queries execution

In [15]:
df["adopted_products"] = df.apply(lambda x: promotion_query(x,"adoption"),axis=1)

In [16]:
df["loyalty1"] = df.apply(lambda x: promotion_query(x,"loyalty1"),axis=1)

In [17]:
df["loyalty2"] = df.apply(lambda x: promotion_query(x,"loyalty2"),axis=1)

In [18]:
df.to_csv(output_file)

In [19]:
items = pd.read_csv("../../datasets/Total/items.csv",index_col=0)
items.ARTICLE_ID.astype(str).replace(mapping,inplace=True)
items.drop(["IDS_LIST"],axis=1,inplace=True)
items.set_index("ARTICLE_ID",inplace=True)

items["adoption_counts"]=0
items["loyalty1_counts"]=0
items["loyalty2_counts"]=0


adoption_counts,loyalty1_counts,loyalty2_counts= [],[],[]
for i,j,k in zip(df.adopted_products.values.tolist(),df.loyalty1.values.tolist(),df.loyalty2.values.tolist()):
    if i!=set():
        adoption_counts+= list(i) 
    if j!=set():
        loyalty1_counts += list(j)
    if k!=set():
        loyalty2_counts += list(k)
adoption_counts, loyalty1_counts,loyalty2_counts = Counter(adoption_counts),Counter(loyalty1_counts),Counter(loyalty2_counts)


for i in adoption_counts:
    items.loc[int(i),"adoption_counts"] = adoption_counts[i]
for j in loyalty1_counts:
    items.loc[int(j),'loyalty1_counts'] = loyalty1_counts[j]
for j in loyalty2_counts:
    items.loc[int(j),'loyalty2_counts'] = loyalty2_counts[j]
    
items = items.sort_values(by=items.columns[-2:].tolist(),ascending=False)
items

,DESCRIPTION,adoption_counts,loyalty1_counts,loyalty2_counts
ARTICLE_ID,,,,
3425909001007,RECHARGE LAVAGE TW - A,96,21,29
3425909002011,RECHARGE LAVAGE TW - D,91,7,11
3425901018140,BIDON AD BLUE 10L,28,2,2
3425901028170,QUARTZ IN LG LIFE 5W30,15,0,1
3019081236250,1 6 RILLETTES THON 125G,0,0,0
...,...,...,...,...
3700619604725,ZIGOH DESO FLEUR FUSHIA,0,0,0
3103220009055,ZIGOTO REGLISSE,0,0,0
3700300542008,ZIP AMIS LUDENDO,0,0,0


In [126]:
df = full_df()
df.shape

(680, 9)

In [73]:
df["loyalty"] = df.apply(lambda x :x["itemsets_x"].intersection(x["itemsets_y"].intersection(x["itemsets_z"])),axis=1)

In [74]:
df["promo_exclusive_products"] = df.apply(lambda x : x["itemsets_y"].difference(x["itemsets_x"]).difference(x["itemsets_z"]),axis=1)

In [75]:
df["replacement"] = df.apply(lambda x : x["itemsets_x"].intersection(x["itemsets_y"]).difference(x["itemsets_z"]),axis=1)

In [76]:
def extract_stats(x,items,columns):
    for i in columns:
        for j in x[i]:
            items.loc[j,i] += 1
columns = ["loyalty","promo_exclusive_products","replacement"]
for i in columns:
    items[i] = 0 
e = df.apply(lambda x : extract_stats(x,items,columns),axis=1)

In [77]:
promo_products = pd.read_csv("../../datasets/Total/promo_products.csv").ARTICLE_ID.unique()
items.loc[promo_products].sort_values(by=columns,ascending=False).head(10)

,DESCRIPTION,adoption_counts,loyalty1_counts,loyalty2_counts,loyalty,promo_exclusive_products,replacement
ARTICLE_ID,,,,,,,
3425909002011,RECHARGE LAVAGE TW - D,91,7,11,0,383,8
3425901018720,TOTAL ACTIVA 9000 5W40 2L,15,0,0,0,120,0
3425901028170,QUARTZ IN LG LIFE 5W30,15,0,1,0,77,0
3425909001007,RECHARGE LAVAGE TW - A,96,21,29,0,61,1
3425901018140,BIDON AD BLUE 10L,28,2,2,0,27,0
3425901014890,ACTIVA 5000 ESS 15W40,0,0,0,0,0,0
3425901019178,ACTIVA 7000E 5L,0,0,0,0,0,0
3425900000962,ACTIVA 9000 1L 5W40,0,0,0,0,0,0
3011449262970,ACTIVA 9000 5W4,0,0,0,0,0,0


In [78]:
res= []
df.user_id_z = df.apply (lambda x : [i for i in list(x["user_id_x"].intersection(x["user_id_y"]))[0].split(',')],axis=1)
df.apply(lambda x : [res.append(i) for i in list(x["user_id_x"].intersection(x["user_id_y"]))[0].split(',')],axis=1)
users = pd.DataFrame(set(res),columns=["CUST_ID"])
users = users.set_index("CUST_ID")

In [81]:
for i in columns:
    users[i]= 0
    users[i]= users[i].apply(lambda x : list())

In [82]:
def extract_stat_by_user(x,users,items,columns):
    for user_id in x["user_id_z"]:
        for i in columns:
            for val in x[i]:
                users.loc[user_id,i].append(items.loc[int(val),"DESCRIPTION"])

e = df.apply(lambda x : extract_stat_by_user(x,users,items,columns),axis=1)
users

,loyalty,promo_exclusive_products,replacement
CUST_ID,,,
101105,[],[BIDON AD BLUE 10L],[]
477520,[],"[RECHARGE LAVAGE TW - A, RECHARGE LAVAGE TW - A]",[]
480255,[],"[RECHARGE LAVAGE TW - A, RECHARGE LAVAGE TW - ...",[]
1170991,[],[QUARTZ IN LG LIFE 5W30],[]
410397,[],"[RECHARGE LAVAGE TW - A, RECHARGE LAVAGE TW - A]",[RECHARGE LAVAGE TW - A]
150303,[],"[RECHARGE LAVAGE TW - D, RECHARGE LAVAGE TW - D]",[]
199984,[],"[BIDON AD BLUE 10L, BIDON AD BLUE 10L, BIDON A...",[]
484507,[],"[RECHARGE LAVAGE TW - D, RECHARGE LAVAGE TW - D]",[]
156096,[],"[RECHARGE LAVAGE TW - D, RECHARGE LAVAGE TW - ...",[]


In [121]:
users_df = pd.read_csv("../../datasets/Total/users.csv",sep=";")
users.index=users.index.astype(int)
stats_df = users.merge(users_df,left_index=True,right_on="CUST_ID")
stats_df.head()

,loyalty,promo_exclusive_products,replacement,CUST_ID,SEX,AGE,DEPARTEMENT
140355,[],[BIDON AD BLUE 10L],[],101105,M,50-65,62
434297,[],"[RECHARGE LAVAGE TW - A, RECHARGE LAVAGE TW - A]",[],477520,M,<35,62
644203,[],"[RECHARGE LAVAGE TW - A, RECHARGE LAVAGE TW - ...",[],480255,F,35-49,59
848702,[],[QUARTZ IN LG LIFE 5W30],[],1170991,F,35-49,38
661410,[],"[RECHARGE LAVAGE TW - A, RECHARGE LAVAGE TW - A]",[RECHARGE LAVAGE TW - A],410397,M,35-49,78


In [122]:
stats_df = stats_df.groupby(["AGE","SEX"]).promo_exclusive_products.apply(sum).apply(set).apply(list).to_frame()
stats_df.to_csv("test.csv")

In [123]:
stats_df

promo_exclusive_products
AGE   SEX                                                   
35-49 F    [RECHARGE LAVAGE TW - D, QUARTZ IN LG LIFE 5W3...
      M    [RECHARGE LAVAGE TW - D, BIDON AD BLUE 10L, RE...
50-65 M    [BIDON AD BLUE 10L, RECHARGE LAVAGE TW - A, RE...
<35   F                             [RECHARGE LAVAGE TW - D]
      M    [RECHARGE LAVAGE TW - D, BIDON AD BLUE 10L, RE...

In [124]:
!cat test.csv

AGE,SEX,promo_exclusive_products
35-49,F,"['RECHARGE LAVAGE TW - D', 'QUARTZ IN LG LIFE 5W30', 'RECHARGE LAVAGE TW - A']"
35-49,M,"['RECHARGE LAVAGE TW - D', 'BIDON AD BLUE 10L', 'RECHARGE LAVAGE TW - A']"
50-65,M,"['BIDON AD BLUE 10L', 'RECHARGE LAVAGE TW - A', 'RECHARGE LAVAGE TW - D', 'QUARTZ IN LG LIFE 5W30', 'TOTAL ACTIVA 9000 5W40 2L']"
<35,F,['RECHARGE LAVAGE TW - D']
<35,M,"['RECHARGE LAVAGE TW - D', 'BIDON AD BLUE 10L', 'RECHARGE LAVAGE TW - A']"


In [112]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(items.DESCRIPTION)

LabelEncoder()

In [114]:
stats_df.promo_exclusive_products = stats_df.promo_exclusive_products.apply(lambda x : le.transform(x))